In [ ]:
!pip install pandas numpy scikit-learn matplotlib
!pip install transformers datasets sentencepiece
!pip install nltk spacy
!python -m spacy download en_core_web_sm
!pip install feedparser
!pip install beautifulsoup4 requests
!pip install torch accelerate
!pip install emoji unidecode html2text
!pip install openai
!pip install faker
!pip install peft bitsandbytes


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 44.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:

# ==================== IMPORTS ====================
import feedparser
import pandas as pd
from datetime import datetime
import os
import time
import requests
from bs4 import BeautifulSoup

# ==================== SCRAPER ====================
def scrape_news_combined(target_count=1000, sleep_sec=1):
    # --- RSS feeds ---
    rss_feeds = {
        'bbc': "http://feeds.bbci.co.uk/news/rss.xml",
        'cnn': "http://rss.cnn.com/rss/edition.rss",
        'aljazeera': "https://www.aljazeera.com/xml/rss/all.xml",
        'dawn': "https://www.dawn.com/feed",
        'tribune': "https://tribune.com.pk/feed",
        'geo': "https://www.geo.tv/rss/1/1",
        'express': "https://www.express.pk/feed/",
        'bbc_sport': "http://feeds.bbci.co.uk/sport/rss.xml?edition=uk",
        'espn': "https://www.espn.com/espn/rss/news",
        'reuters': "http://feeds.reuters.com/reuters/topNews"
    }

    # --- Direct website pages (multi-page scraping) ---
    website_pages = {
        'bbc_news': "https://www.bbc.com/news",
        'cnn_news': "https://edition.cnn.com/world",
        'aljazeera_news': "https://www.aljazeera.com/news/",
        'dawn_news': "https://www.dawn.com/latest-news",
        'tribune_news': "https://tribune.com.pk/latest",
        'geo_news': "https://www.geo.tv/latest",
        'express_news': "https://www.express.pk/latest",
        'bbc_sport_news': "https://www.bbc.com/sport",
        'espn_news': "https://www.espn.com/news/",
        'reuters_news': "https://www.reuters.com/news/world"
    }

    headlines = []

    # Load existing headlines
    file_path = "data/scraped_headlines.csv"
    if os.path.exists(file_path):
        existing_df = pd.read_csv(file_path)
        existing_texts = set(existing_df['text'].values)
    else:
        existing_df = pd.DataFrame()
        existing_texts = set()

    total_collected = len(existing_texts)
    print(f"Already have {total_collected} headlines")

    # ===== Step 1: RSS Scraping =====
    for source, feed_url in rss_feeds.items():
        if total_collected >= target_count:
            break
        try:
            feed = feedparser.parse(feed_url)
            new_count = 0
            for entry in feed.entries:
                title = entry.title.strip()
                if len(title) > 10 and title not in existing_texts:
                    headlines.append({
                        'text': title,
                        'source': source,
                        'date': datetime.now().strftime('%Y-%m-%d'),
                        'sentiment': 'unlabeled'
                    })
                    existing_texts.add(title)
                    total_collected += 1
                    new_count += 1
                    if total_collected >= target_count:
                        break
            print(f"✅ RSS Scraped {new_count} headlines from {source}")
            time.sleep(sleep_sec)
        except Exception as e:
            print(f"❌ RSS Error {source}: {e}")

    # ===== Step 2: Direct website scraping (first 5 pages) =====
    for source, url in website_pages.items():
        if total_collected >= target_count:
            break
        new_count = 0
        for page in range(1, 6):
            if total_collected >= target_count:
                break
            try:
                page_url = f"{url}?page={page}" if '?' not in url else f"{url}&page={page}"
                res = requests.get(page_url, timeout=10)
                soup = BeautifulSoup(res.content, "html.parser")
                tags = soup.find_all(['h1', 'h2', 'h3'])
                for tag in tags:
                    title = tag.get_text().strip()
                    if len(title) > 10 and title not in existing_texts:
                        headlines.append({
                            'text': title,
                            'source': source,
                            'date': datetime.now().strftime('%Y-%m-%d'),
                            'sentiment': 'unlabeled'
                        })
                        existing_texts.add(title)
                        total_collected += 1
                        new_count += 1
                        if total_collected >= target_count:
                            break
                time.sleep(sleep_sec)
            except Exception as e:
                print(f"❌ Error scraping {source} page {page}: {e}")
        print(f"✅ Web Scraped {new_count} headlines from {source}")

    # ===== Step 3: Save combined dataset =====
    os.makedirs("data", exist_ok=True)
    new_df = pd.DataFrame(headlines)
    combined_df = pd.concat([existing_df, new_df], ignore_index=True)
    combined_df = combined_df.drop_duplicates(subset=['text']).reset_index(drop=True)
    combined_df.to_csv(file_path, index=False)
    print(f"\n🎉 Scraping done! Total unique headlines now: {len(combined_df)}")

    return combined_df

# ==================== RUN ====================
df = scrape_news_combined(target_count=1000)
df.head(10)


Already have 997 headlines
✅ RSS Scraped 0 headlines from bbc
✅ RSS Scraped 0 headlines from cnn
✅ RSS Scraped 0 headlines from aljazeera
✅ RSS Scraped 0 headlines from dawn
✅ RSS Scraped 0 headlines from tribune
✅ RSS Scraped 0 headlines from geo
✅ RSS Scraped 0 headlines from express
✅ RSS Scraped 0 headlines from bbc_sport
✅ RSS Scraped 2 headlines from espn
✅ RSS Scraped 0 headlines from reuters
✅ Web Scraped 1 headlines from bbc_news

🎉 Scraping done! Total unique headlines now: 1000


,text,source,date,sentiment
0,Ukraine prepares new peace plan as Zelensky ru...,bbc,2025-12-09,unlabeled
1,Ukrainians raise flag to show BBC the fight go...,bbc,2025-12-09,unlabeled
2,Pioneering new UK treatment reverses incurable...,bbc,2025-12-09,unlabeled
3,UN environment report 'hijacked' by US and oth...,bbc,2025-12-09,unlabeled
4,Why are Thailand and Cambodia fighting at the ...,bbc,2025-12-09,unlabeled
5,Dozens injured after magnitude 7.5 quake strik...,bbc,2025-12-09,unlabeled
6,Australia's social media ban for children has ...,bbc,2025-12-09,unlabeled
7,CCTV shows Matisse heist suspects making off w...,bbc,2025-12-09,unlabeled
8,"Ben & Jerry's brand could be destroyed, says c...",bbc,2025-12-09,unlabeled
9,Fact-checking Trump’s denial over releasing Ve...,bbc,2025-12-09,unlabeled


In [ ]:
import pandas as pd
from google.colab import files

# 1. Read the CSV file directly
# Make sure "scraped_headlines.csv" is in the Files tab on the left in Colab
filename = "data/scraped_headlines.csv"

try:
    df = pd.read_csv(filename)
    print(f"Successfully loaded {filename} with {len(df)} rows.")
except FileNotFoundError:
    print(f"Error: Could not find '{filename}'. Please check the spelling or ensure the file is in the main directory.")

# 2. Define the Sentiment Logic and Keywords
positive_words = [
    "peace", "win", "victory", "success", "breakthrough", "recover", "safe", "rescue",
    "new", "gain", "profit", "best", "innovative", "improving", "solution", "agreement",
    "treaty", "love", "celebrate", "award", "honor", "upgrade", "growth", "boost",
    "support", "help", "promising", "hope", "benefit", "good", "great", "lead", "leader",
    "progress", "triumph", "restore", "wonderful", "brilliant", "joy", "strong"
]

negative_words = [
    "war", "fight", "conflict", "kill", "dead", "death", "injured", "injury", "crisis",
    "disaster", "attack", "bomb", "destroy", "damage", "loss", "fail", "bad", "worst",
    "crash", "stolen", "heist", "murder", "fear", "worry", "concern", "ban", "scramble",
    "arrest", "charge", "jail", "prison", "fire", "quake", "flood", "warn", "threat",
    "risk", "danger", "deadly", "fatal", "victim", "violence", "crime", "shortage",
    "pollution", "hijack", "terror", "tragedy", "accident", "emergency", "collapse",
    "strike", "protest", "riot", "dispute", "tension", "suspect", "guilty", "court",
    "lawsuit", "suffer", "pain", "ill", "sick", "virus", "disease", "cancer", "outbreak"
]

neutral_words = []

def auto_label_lexicon(text):
    if not isinstance(text, str):
        return "neutral"

    text_lower = text.lower()

    # Calculate scores based on keyword presence
    pos_count = sum(word in text_lower for word in positive_words)
    neg_count = sum(word in text_lower for word in negative_words)

    if pos_count > neg_count:
        return "positive"
    elif neg_count > pos_count:
        return "negative"
    else:
        return "neutral"

# 3. Apply the labeling function
if 'df' in locals():
    # Ensure we are targeting the correct column
    target_column = 'text' if 'text' in df.columns else df.columns[0]
    print(f"Labeling based on column: '{target_column}'")

    df['sentiment'] = df[target_column].apply(auto_label_lexicon)

    # 4. Save and Download the result
    output_filename = "labeled_headlines.csv"
    df.to_csv(output_filename, index=False)

    print("\nProcessing complete. Downloading result...")
    files.download(output_filename)

    # Display a preview
    print("\nPreview of labeled data:")
    print(df[[target_column, 'sentiment']].head().to_markdown(index=False))

Successfully loaded data/scraped_headlines.csv with 1000 rows.
Labeling based on column: 'text'

Processing complete. Downloading result...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


Preview of labeled data:
| text                                                                                    | sentiment   |
|:----------------------------------------------------------------------------------------|:------------|
| Ukraine prepares new peace plan as Zelensky rules out giving up land                    | positive    |
| Ukrainians raise flag to show BBC the fight goes on in city claimed by Russia           | negative    |
| Pioneering new UK treatment reverses incurable blood cancer in some patients            | neutral     |
| UN environment report 'hijacked' by US and others over fossil fuels, top scientist says | negative    |
| Why are Thailand and Cambodia fighting at the border?                                   | negative    |


In [ ]:
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import nltk

# Download stopwords once
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
stop_words = set(stopwords.words('english'))
stemmer = SnowballStemmer('english')

def preprocess_news(text):
    text = str(text).lower()  # lowercase

    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text)

    # Remove special characters (keep letters, numbers, spaces)
    text = re.sub(r'[^a-z0-9\s]', ' ', text)

    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()

    # Remove stopwords
    words = [word for word in text.split() if word not in stop_words]

    # Optional: stemming
    words = [stemmer.stem(word) for word in words]

    return ' '.join(words)


In [ ]:
# ==================== IMPORTS ====================
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import os

# ==================== DOWNLOAD NLTK DATA ====================
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

# ==================== SETUP ====================
# Standard stopwords minus important negations
negations = {'not', 'no', 'never'}
stop_words = set(stopwords.words('english')) - negations

lemmatizer = WordNetLemmatizer()

# ==================== PREPROCESS FUNCTION ====================
def preprocess_news(text):
    text = str(text).lower()  # lowercase

    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text)

    # Remove special characters (keep letters, numbers, spaces)
    text = re.sub(r'[^a-z0-9\s]', ' ', text)

    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()

    # Remove stopwords (keep negations) and lemmatize
    words = [lemmatizer.lemmatize(word) for word in text.split() if word not in stop_words]

    return ' '.join(words)

# ==================== LOAD & PREPROCESS ====================
# Create 'data' folder if it doesn't exist
os.makedirs("data", exist_ok=True)

# Load your labeled CSV
df = pd.read_csv("labeled_headlines.csv")

# Preprocess news
df['cleaned_text'] = df['text'].apply(preprocess_news)

# Remove empty rows after cleaning
df = df[df['cleaned_text'].str.len() > 0].reset_index(drop=True)

# Save preprocessed CSV
df.to_csv("data/preprocessed_news.csv", index=False)
print(f"✅ Preprocessing done! Total samples: {len(df)}")

# Show first 10 rows
df.head(10)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


✅ Preprocessing done! Total samples: 892


,text,source,date,sentiment,cleaned_text
0,Ukraine prepares new peace plan as Zelensky ru...,bbc,2025-12-09,positive,ukraine prepares new peace plan zelensky rule ...
1,Ukrainians raise flag to show BBC the fight go...,bbc,2025-12-09,negative,ukrainian raise flag show bbc fight go city cl...
2,Pioneering new UK treatment reverses incurable...,bbc,2025-12-09,neutral,pioneering new uk treatment revers incurable b...
3,UN environment report 'hijacked' by US and oth...,bbc,2025-12-09,negative,un environment report hijacked u others fossil...
4,Why are Thailand and Cambodia fighting at the ...,bbc,2025-12-09,negative,thailand cambodia fighting border
5,Dozens injured after magnitude 7.5 quake strik...,bbc,2025-12-09,negative,dozen injured magnitude 7 5 quake strike north...
6,Australia's social media ban for children has ...,bbc,2025-12-09,negative,australia social medium ban child left big tec...
7,CCTV shows Matisse heist suspects making off w...,bbc,2025-12-09,negative,cctv show matisse heist suspect making artwork...
8,"Ben & Jerry's brand could be destroyed, says c...",bbc,2025-12-09,negative,ben jerry brand could destroyed say co founder
9,Fact-checking Trump’s denial over releasing Ve...,bbc,2025-12-09,negative,fact checking trump denial releasing venezuela...


 **Learning and experimenting:Training model on my scrapped dataset**


In [ ]:
!pip install pandas numpy tensorflow --quiet


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau


In [ ]:
# Load your preprocessed CSV
df = pd.read_csv("data/preprocessed_news.csv")

# Encode sentiment labels
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['sentiment'])

print(f"Classes mapping: {dict(zip(label_encoder.classes_, range(len(label_encoder.classes_))))}")

X = df['cleaned_text'].values
y = df['label'].values

# Train-validation-test split
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp
)

print(f"Train: {len(X_train)}, Val: {len(X_val)}, Test: {len(X_test)}")


Classes mapping: {'negative': 0, 'neutral': 1, 'positive': 2}
Train: 624, Val: 134, Test: 134


In [ ]:
MAX_WORDS = 10000
MAX_LEN = 100

tokenizer = Tokenizer(num_words=MAX_WORDS, oov_token='<OOV>')
tokenizer.fit_on_texts(X_train)

# Convert texts to sequences
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_val_seq = tokenizer.texts_to_sequences(X_val)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Pad sequences
X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_LEN, padding='post', truncating='post')
X_val_pad = pad_sequences(X_val_seq, maxlen=MAX_LEN, padding='post', truncating='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=MAX_LEN, padding='post', truncating='post')


In [ ]:
num_classes = len(label_encoder.classes_)

model = Sequential([
   Embedding(input_dim=MAX_WORDS, output_dim=128, input_shape=(MAX_LEN,)),
    Bidirectional(LSTM(128, return_sequences=True)),
    Dropout(0.3),
    Bidirectional(LSTM(64)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(num_classes, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:100: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 100, 128)       │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 100, 256)       │       263,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 100, 256)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ (None, 128)            │       164,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │           195 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,715,971 (6.55 MB)

 Trainable params: 1,715,971 (6.55 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
callbacks = [
    EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-6)
]

history = model.fit(
    X_train_pad, y_train,
    validation_data=(X_val_pad, y_val),
    batch_size=32,
    epochs=20,
    callbacks=callbacks,
    verbose=1
)


Epoch 1/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 33s 836ms/step - accuracy: 0.5385 - loss: 0.9988 - val_accuracy: 0.6045 - val_loss: 0.9181 - learning_rate: 0.0010
Epoch 2/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 10s 473ms/step - accuracy: 0.5887 - loss: 0.9312 - val_accuracy: 0.6119 - val_loss: 0.8739 - learning_rate: 0.0010
Epoch 3/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 11s 542ms/step - accuracy: 0.6937 - loss: 0.7565 - val_accuracy: 0.7090 - val_loss: 0.9274 - learning_rate: 0.0010
Epoch 4/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 11s 545ms/step - accuracy: 0.8392 - loss: 0.3701 - val_accuracy: 0.7090 - val_loss: 1.1171 - learning_rate: 0.0010
Epoch 5/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 11s 538ms/step - accuracy: 0.8614 - loss: 0.2956 - val_accuracy: 0.7015 - val_loss: 1.1066 - learning_rate: 5.0000e-04


In [ ]:
test_loss, test_acc = model.evaluate(X_test_pad, y_test)
print(f"Test Accuracy: {test_acc:.4f}")


5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 112ms/step - accuracy: 0.6076 - loss: 0.9069
Test Accuracy: 0.6119


In [ ]:
import pickle
import os

os.makedirs("model", exist_ok=True)

model.save("model/custom_sentiment_model.h5")
with open("model/tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)
with open("model/label_encoder.pkl", "wb") as f:
    pickle.dump(label_encoder, f)

print("✅ Model and artifacts saved!")


✅ Model and artifacts saved!


In [ ]:
def predict_sentiment(text):
    cleaned_seq = tokenizer.texts_to_sequences([text])
    padded_seq = pad_sequences(cleaned_seq, maxlen=MAX_LEN, padding='post')
    pred = model.predict(padded_seq)[0]
    class_idx = np.argmax(pred)
    return {
        'sentiment': label_encoder.classes_[class_idx],
        'confidence': float(pred[class_idx])
    }

# Test
predict_sentiment("he has eaten an apple")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step


{'sentiment': 'neutral', 'confidence': 0.7100611329078674}